In [1]:
%%capture
%run regression.ipynb

In [2]:
%pip install -q tensorflow

Note: you may need to restart the kernel to use updated packages.


In [18]:
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [19]:
# prepare inverse data (nm -> V)
# Input: current positions (x_out, z_out)
# Output: voltage windows that led to those positions
X_inverse = xz_data_out  # (N, 2) - positions
y_inverse = xz_data_in   # (N, 6) - voltage windows
X_train, X_test, y_train, y_test = train_test_split(
     X_inverse, y_inverse, test_size=0.2, random_state=42
)


WINDOW_LEN = 3 # using the same window length as in regression
print(X_inverse.shape)
input_dim = X_inverse.shape[1]  # 2 (x and z positions)
output_dim = y_inverse.shape[1]  # 6 (3 x-voltage + 3 z-voltage windows)
print(input_dim, output_dim)

(359999, 2)
2 6


In [20]:
# try a simple LinearRegression model first
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin = lin_reg.predict(X_test)
r2_lin = r2_score(y_test, y_pred_lin)
r2_lin_weighted = r2_score(y_test, y_pred_lin, multioutput="variance_weighted")
print(f"Linear Regression R2: {r2_lin:.4f}")
print(f"Linear Regression R2 variance weighted: {r2_lin_weighted:.4f}")

Linear Regression R2: 0.3770
Linear Regression R2 variance weighted: 0.3924


The feedforward (voltage -> position) R2 score came out to **0.74** (which is good).
The inverse of this (position -> voltage) gives us an R2 score of **0.39**. This means the inverse mapping is fundamentally harder.

In [6]:
# scale the data for the NN
X_scaled = StandardScaler().fit_transform(X_inverse)
y_scaled = StandardScaler().fit_transform(y_inverse)

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42
)

**APPROACH 1**

Training a NN with just positions and predicting the voltage window.

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y_train_scaled.shape[1])
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_split=0.2,
    epochs=500,
    batch_size=128,
    verbose=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=30,
            restore_best_weights=True
        ),
    ]
)

y_pred = model.predict(X_test_scaled, verbose=0)
r2 = r2_score(y_test_scaled, y_pred)
r2_weighted = r2_score(y_test_scaled, y_pred, multioutput="variance_weighted")

print(f"Overall R2 score: {r2:.4f}")
print(f"Overall R2 score weighted: {r2_weighted:.4f}")

Overall R2 score: 0.4188
Overall R2 score weighted: 0.4192


**APPROACH 2**

Create a feature vector containing the current state and previous 2 states
 
 `[ x(t−2), x(t−1), x(t), z(t−2), z(t−1), z(t) ]`

This way the network has more information about the previous states - not just the current state

In [15]:
x_out = df["x_out"].values.reshape(-1, 1)
z_out = df["z_out"].values.reshape(-1, 1)

x_out_window = sliding_window_view(x_out, WINDOW_LEN, 0).squeeze()
z_out_window = sliding_window_view(z_out, WINDOW_LEN, 0).squeeze()

xz_out_window = np.concatenate([x_out_window, z_out_window], axis=1)

print(f"Original input (position only): {xz_data_out.shape}")
print(f"New input (position history): {xz_out_window.shape}")

# Now input and output have same length and meaning
X_inverse_history = xz_out_window  # (N, 6) - position windows
y_inverse = xz_data_in             # (N, 6) - voltage windows

Original input (position only): (359999, 2)
New input (position history): (359999, 6)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
     X_inverse_history, y_inverse, test_size=0.2, random_state=42
)

In [17]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin = lin_reg.predict(X_test)
r2_lin = r2_score(y_test, y_pred_lin)
r2_lin_weighted = r2_score(y_test, y_pred_lin, multioutput="variance_weighted")
print(f"Linear Regression R2: {r2_lin:.4f}")
print(f"Linear Regression R2 variance weighted: {r2_lin_weighted:.4f}")

Linear Regression R2: 0.4847
Linear Regression R2 variance weighted: 0.4980


In [13]:
X_scaled = StandardScaler().fit_transform(X_inverse_history)
y_scaled = StandardScaler().fit_transform(y_inverse)

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
    X_scaled, y_scaled, test_size=0.2, random_state=42
)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y_train_scaled.shape[1])
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_split=0.2,
    epochs=500,
    batch_size=128,
    verbose=0,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=30,
            restore_best_weights=True
        ),
    ]
)

y_pred = model.predict(X_test_scaled, verbose=0)
r2 = r2_score(y_test_scaled, y_pred)
r2_weighted = r2_score(y_test_scaled, y_pred, multioutput="variance_weighted")

print(f"Overall R2 score: {r2:.4f}")
print(f"Overall R2 score weighted: {r2_weighted:.4f}")

Overall R2 score: 0.6311
Overall R2 score weighted: 0.6313
